In [ ]:
import openai
import langchain
import pinecone

from langchain.document_loaders import PyPDFDirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import Pinecone


In [ ]:
from langchain.llms import OpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

In [ ]:

def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents= file_loader.load()
    return documents

In [ ]:
doc = read_doc('documents/')
doc


In [ ]:

def chunk_data(docs,chunk_size =800,overlap =50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,chunk_overlap = overlap)
    docs = text_splitter.split_documents(docs)
    return docs

In [ ]:
chunk_data(doc)


In [ ]:
len(doc)

In [ ]:
#embedding technique of openai
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embeddings

In [ ]:
vectors = embeddings.embed_query('how  are u')
vectors

In [ ]:
len(vectors)

In [ ]:
#vector search_db 


In [ ]:
from dotenv import load_dotenv
import os

from pinecone import Pinecone

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(
    api_key=PINECONE_API_KEY
    )
index = pc.Index("langchainbudget")

In [ ]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=doc,
    embedding=embeddings,
    index_name="langchainbudget"  # <-- pass the name as string
)


In [ ]:
#cosine similarity from vector DB
def retrieve_query(query,k=1):
    matching_results = vectorstore.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain_community import Groq

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",  
    groq_api_key=GROQ_API_KEY,
    temperature=0.2
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # can also use "refine" or "map_reduce"
)

query = "What is agricultural budget?"
answer = qa_chain.run(query)
print(answer)
